In [3]:
import geopandas as gpd
import pandas as pd
from pandas.io.json import json_normalize
import urllib.request
import json
from geopandas import GeoDataFrame
from shapely.geometry import Point
from datetime import datetime, timedelta

In [4]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [5]:
def convert_coordinates(coordinates, srcEPSG, dstEPSG):
    """
    Converts points to coordinatesystem
    
    Input:
    coordinates: (list of) tuples
    """    
    from osgeo.osr import SpatialReference, CoordinateTransformation

    src = SpatialReference()
    dst = SpatialReference()
    
    src.ImportFromEPSG(srcEPSG)
    dst.ImportFromEPSG(dstEPSG)
    
    transform = CoordinateTransformation(src, dst)

    if not type(coordinates) is list:
        coordinates = [(coordinates)]
       
    if len(coordinates[0]) == 2:
        dimension = 2
    else:
        dimension = 3
       
    transformed = []
    for crd in coordinates:
        transformed.append((transform.TransformPoint(*crd)))
    
    if dimension == 2:
        transformed = [(i[0], i[1]) for i in transformed]
    return transformed

In [6]:
# set timedelta in epoch
min_timedelta = 0
time_epoch = int((datetime.now() - timedelta(minutes=min_timedelta)).timestamp())
print ((datetime.now() - timedelta(minutes=min_timedelta)).strftime('%c'))
print (time_epoch)

Mon Jun  5 22:07:10 2017
1496693230


In [14]:
# urls to retrieve AIS data from data providers
#url_marinetraffic  = 'https://www.marinetraffic.com/getData/get_data_json_4/z:5/X:7/Y:4/station:0'
minlat = str(52.316734)
maxlat = str(52.476281)
minlon = str(4.422955)
maxlon = str(5.059489)
timecode = str(time_epoch)
mmsi = str(244660778)
zoom = str(10)

url_vesselsonmap = 'http://www.myshiptracking.com/requests/vesselsonmap.php?type=json&minlat='+minlat+'&maxlat='+maxlat+'&minlon='+minlon+'&maxlon='+maxlon+'&zoom='+zoom+'&mmsi='+mmsi+'&timecode='+timecode
print (url_vesselsonmap)

http://www.myshiptracking.com/requests/vesselsonmap.php?type=json&minlat=52.316734&maxlat=52.476281&minlon=4.422955&maxlon=5.059489&zoom=10&mmsi=244660778&timecode=1496693230


In [15]:
# request the tracking url and decode as json and parse to pandas dataframe
r = urllib.request.urlopen(url_vesselsonmap)
data = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))
df_vesselsonmap = json_normalize(data[0]['DATA'])

In [16]:
# drop all the NaNs
# and convert strings to floats
df_vesselsonmap.dropna(inplace=True)
df_vesselsonmap.LNG = df_vesselsonmap.LNG.astype(float)
df_vesselsonmap.LAT = df_vesselsonmap.LAT.astype(float)

In [17]:
## convert LNG and LAT columns to geodataframe
# geometry = [Point(xy) for xy in zip(df.LNG.astype(float), df.LAT.astype(float))]
# df = df.drop(['LNG', 'LAT'], axis=1)
# crs = {'init': 'epsg:4326'}
# geo_df = GeoDataFrame(df, crs=crs, geometry=geometry)

In [18]:
df_vesselsonmap.head()

,ARV,COG,DEST,IMO,LAT,LNG,MMSI,NAME,R,S1,S2,S3,S4,SOG,T,TYPE,T_OLD
16,,0,,,52.64417,5.09833,2442024,,1496693240,,,,,0,1496693240,11,1496693240
21,5/27/2017 1:07:54 PM,511,URK,0,52.65949,5.59831,1879467,PRINCESS H,1496693234,409,7,1,3,0,1496693234,0,1496693234
27,,511,2,0,52.61185,5.75925,244690032,INSPE 4,1496693235,65,15,5,5,0,1496693235,7,1496693235
40,,0,,,52.58667,4.26028,992446022,WTG_13_PAWP,1496693242,56,56,56,56,0,1496693242,13,1496693242
48,6/2/2017 11:57:30 AM,511,ALPHEN AAN DE RIJN,0,52.56558,4.74113,244820372,LEEUW,1496693237,46,9,3,4,0,1496693237,7,1496693237


In [19]:
import folium

mapa = folium.Map([52.419052, 4.800132],
                  zoom_start=int(zoom),
                  tiles='cartodbpositron')

fg=folium.FeatureGroup(name="ships")
#points = folium.features.GeoJson(df)

for lat,lon,name,dest in zip(df_vesselsonmap['LAT'],df_vesselsonmap['LNG'],df_vesselsonmap['NAME'],df_vesselsonmap['DEST']):
    #print (lat,lon,name,dest)
    try:
        fg.add_child(folium.Marker(location=[lat,lon],popup=(folium.Popup('NAME: '+name+'  '+'DEST: '+dest))))
    except:
        continue

mapa.add_child(fg)
mapa.add_child(folium.LayerControl())

In [20]:
# REQUEST SINGLE SHIP INFORMATION

In [70]:
def getVesselDetails(mmsi):
    # urls to retrieve AIS data from data providers
    # mmsi (type: string)

    url_vesseldetails = 'http://www.myshiptracking.com/requests/vesseldetails.php?type=json&mmsi='+mmsi
    #print (url_vesseldetails)

    # request the tracking url and decode as json and parse to pandas dataframe
    r = urllib.request.urlopen(url_vesseldetails)
    data = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))
    df_vesseldetails = json_normalize(data)
    
    # define width and height of boat
    try:
        df_vesseldetails['V.LENGTH'] = float(df_vesseldetails['V.TO_BOW']) + float(df_vesseldetails['V.TO_STERN'])
        df_vesseldetails['V.WIDTH']  = float(df_vesseldetails['V.TO_STARBOARD']) + float(df_vesseldetails['V.TO_PORT'])
    except Exception as e:
        print (e)
        
    # define RD coordinates of boat
    try:
        rd_coords = convert_coordinates([(float(df_vesseldetails['V.LNG']),float(df_vesseldetails['V.LAT']))], 4326, 28992)[0]
        #print (rd_coords)
        df_vesseldetails['V.RD_X'] = rd_coords[0]
        df_vesseldetails['V.RD_Y'] = rd_coords[1]
    except Exception as e:
        print (e)
    return df_vesseldetails

In [71]:
# get single mmsi to get structure of the series
mmsi = str(df_vesselsonmap['MMSI'].iloc[0])
print (mmsi)
base_df_vDetails = getVesselDetails(mmsi)

2442024
could not convert string to float: 


In [72]:
base_df_vDetails.T

,0
P,[]
TZ,
V.COG,0
V.DESTINATION,
V.DRAUGHT,
V.ETA,
V.FLAG,
V.LAT,52.64417
V.LDEST,
V.LDESTT,


In [73]:
for mmsi in df_vesselsonmap['MMSI'].iloc[1::]:
    print (str(mmsi))
    df_vDetails = getVesselDetails(str(mmsi))
    # concatenate all vessel information to one single dataframe 
    # its slow method as it is copying over the dataframe each time it concatentates..
    base_df_vDetails = pd.concat([base_df_vDetails, df_vDetails], axis=0)

1879467
244690032
992446022
244820372
244670871
244830861
could not convert string to float: 
244810983
244010915
244710631
211558760
244700485
210072000
2444012
205531290
244710660
244820297
372580000
249878
2444010
205205217
205245890
205410590
211386900
211509670
244700716
227654770
244010963
244670518
244130271
244142000
244615958
244670604
244820298
244690914
244700178
244700601
244700853
244870223
205424690
244010988
244020560
244020823
244050085
244660563
244750374
244770199
244890030
244615365
244620144
244660229
244660588
244690646
244670489
244750261
244110236
244110865
244660796
244700102
244700349
244690144
244780406
244690810
244810551
244770939
could not convert string to float: 
244630110
244740575
244690490
244710068
244690055
215964000
244670929
245136000
2442006
244660682
244620984
244670599
244670845
244710198
244700972
269056766
244740851
244870678
244650947
244660778


In [74]:
base_df_vDetails.T

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
P,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
TZ,,,,,,,,,,,...,,,,,,,,,,
V.COG,0,511,511,0,511,511,511,20.1,16,13.8,...,511,511,511,511,511,144.1,511,511,511,511
V.DESTINATION,,URK,,,,,,,,ESD,...,,,,,,,,HUIS_____,,
V.DRAUGHT,,0.2,1,,0.2,1,,,1.5,0,...,0,0.3,2.4,0.2,2.5,0,0,0.2,2.5,0.3
V.ETA,,,,,,,,,,,...,,,,,,,,,,
V.FLAG,,Unknown[00],Netherlands[NL],Unknown[00],Netherlands[NL],Netherlands[NL],,Netherlands[NL],Netherlands[NL],Netherlands[NL],...,Netherlands[NL],Netherlands[NL],Netherlands[NL],Netherlands[NL],Netherlands[NL],Switzerland[CH],Netherlands[NL],Netherlands[NL],Netherlands[NL],Netherlands[NL]
V.LAT,52.64417,52.65945,52.61183,52.58667,52.56559,52.48859,52.56617,52.53817,52.61268,52.523,...,52.12869,52.11064,52.11946,52.12403,52.1133,52.11439,52.09287,52.13062,52.06451,51.96676
V.LDEST,,MEPPEL,0,,ALPHEN AAN DE RI,WOP,,,COAST GUARD DUTIES,ANTW - BREE,...,GOTTERWICKERSHAM !,DEN HAAG,"IJMUIDEN ""50",NIGTEVECHT,ANTWERPEN 12,AMERIKAHAVEN,,,COSTA DEL!G 24TON,MAASTRIC
V.LDESTT,,2015-02-21 08:43,2017-06-05 18:03,,2017-05-30 18:53,2017-04-17 21:03,,,2015-10-17 09:45,2016-07-18 20:33,...,2017-05-21 00:26,2017-06-02 10:46,2015-11-27 07:26,2017-05-20 10:22,2016-05-03 03:41,2017-06-05 11:05,,,2017-03-12 09:41,2017-03-21 10:19


In [12]:
# START CREATING THE NODE XML FILE

In [ ]:
# lat/lon coordinates in 4326
in_points = [(52.379972,4.958865),(51.967049,5.320434),(52.466740,4.606085),(52.014135,5.110308),(52.036883,5.112706),(52.374937,4.948080),(52.420181,4.809713),(52.396220,4.813572),(52.521354,3.521670),(53.036403,4.142134),(52.075505,2.839227)]

# corresponding nodes
loc_codes = ['OSZ','PIZ','SIJ','PBS','LKD','GMC','HPT','HHN','NZW','NZN','NZZ']

# convert to rd coordinates (28992)
c = []
for b in in_points: c.append(b[::-1])
rd_coords = convert_coordinates(c, 4326, 28992)

In [ ]:
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment

In [ ]:
# CREATE XML NODES
nodes = Element('nodes')
for idx in range(len(loc_codes)):
    #print (idx)
    loc = loc_codes[idx]
    x   = str(rd_coords[idx][0])
    y   = str(rd_coords[idx][1]    )
    #print (x,y,loc) 
    child_1 = SubElement(nodes, 'node')
    child_1.set('id',loc)
    child_1.set('x',x)
    child_1.set('y',y)
print(prettify(nodes))

In [ ]:
# END CREATING THE NODE XML FILE

In [ ]:
df.T

In [ ]:
data.V.TO_BOW,data.V.TO_STERN,data.V.TO_STARBOARD,data.V.TO_PORT

In [ ]:
function format_size(a, b, c, d) {
    if (isNaN(parseFloat(a) + parseFloat(b))) {
        return "---";
    } else {
        return (parseFloat(a) + parseFloat(b)) + " x " + (parseFloat(c) + parseFloat(d)) + " " + lang_files['m'];